In [6]:
#修改in_total表无id企业名

#com_origin：404企业原名
#com_search: 正确企业名

from DBUtils.PooledDB import PooledDB
import pymysql

_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=44,
                maxconnections=512, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '39.105.9.20',
            "port": 3306,
            "user": 'root',
            "passwd": 'bigdata_oil',
            "db": 'cxd_test',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()

connection = get_dbservice_mysql_conn()


# conn = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
# conn = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_test',charset = 'utf8')

com_search = []

def update_in_total_name(com_origin, com_correct):
#     connection = conn
    with connection.cursor() as cursor:
        sql = "update `in_total` set `com_name`='{0}' where com_name='{1}'".format(com_correct,com_origin)
        cursor.execute(sql)
    connection.commit()
    connection.close()

# com_search.append(name)

# connection = conn
sql = "SELECT `com_id`,`com_name` FROM `in_total` WHERE `com_id` is null"
with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
com_origin = [com[1] for com in result]

for com_name in com_origin:
    if com_name.endswith('科目余额表'):
        name = com_name[:-5]
        update_in_total_name(com_name, name)
    elif com_name.endswith('序时账'):
        name = com_name[:-3]
        update_in_total_name(com_name, name)
    elif com_name.endswith('19'):
        name = com_name[:-2]
        update_in_total_name(com_name, name)
    else: 
        name = com_name
    com_search.append(name)
            
# def get_com_id(name):
#     connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')

# #     name = '大连大昌石油化工有限公司'

#     sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` ='%s'" %name
#     with connection.cursor() as cursor:
#         cursor.execute(sql)
#         result = cursor.fetchone()
#     return result[0]

def update_in_total_id(com_name):
#     print(com_name)
#     connection = conn
    with connection.cursor() as cursor:
        sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` ='%s'" %com_name
        print(sql)
        cursor.execute(sql)
        result = cursor.fetchone()
    com_id = result[0]
    with connection.cursor() as cursor:
        sql = "update `in_total` set `com_id`='{0}' where com_name='{1}'".format(com_id,com_name)
        print(sql)
        cursor.execute(sql)
    connection.commit()
    connection.close()

for com in com_search:
    if len(com)<4:
        continue
    connection = get_dbservice_mysql_conn()
    update_in_total_id(com)
print('Done')

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='福建省武夷山市竹柴炭公司好美家快捷酒店'


TypeError: 'NoneType' object is not subscriptable